In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import errno
import json
import pickle
import glob
import multiprocessing
from time import time  # To time our operations
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

print('import done')

import done


# Data Cleaning and Preprocessing

In [2]:
def clean(txt):
    """
    Basic string loading code.

    :param txt:
    :return:
    """
    txt = re.sub(r'.\n+', '. ', txt)  # replace multiple newlines with period
    txt = re.sub(r'\n+', '', txt)  # replace multiple newlines with period
    txt = re.sub(r'\[\d+\]', ' ', txt)  # remove reference numbers
    txt = re.sub(' +', ' ', txt)
    
    txt = re.sub(',', ' ', txt)
    txt = re.sub(r'\([^()]*\)', '', txt)
    txt = re.sub(r'https?:\S+\sdoi', '', txt)
    txt = re.sub(r'biorxiv', '', txt)
    txt = re.sub(r'preprint', '', txt)
    txt = re.sub(r':', ' ', txt)
    return txt.lower()



In [3]:
class document():
    def __init__(self, file_path):
        if file_path:
            with open(file_path) as file:
                data = json.load(file)
                self.paper_id = data['paper_id']
                self.title = data['metadata']['title']
                self.abstract_tripples = {}
                self.text_tripples = {}
                self.key_phrases = ""
                self.abstract = ""
                self.text = ""
                self.entities = {}
                if 'abstract' in data:
                    for section in data['abstract']:
                        self.abstract = self.abstract + "\n" + section["text"]

                for section in data['body_text']:
                    self.text = self.text + "\n" + section['text']

    def clean_text(self):
        self.abstract = clean(self.abstract)
        self.text = clean(self.text)
        self.title =clean(self.title)
        final_data_dict = self.combine_data()
        return final_data_dict

    def combine_data(self):
        self.data = {'paper_id': self.paper_id,
                     'title': self.title,
                     'abstract': self.abstract,
                     'text': self.text,
                     'abstract_tripples': self.abstract_tripples,
                     'text_tripples': self.text_tripples,
                     'key_phrases': self.key_phrases,
                     'entities': self.entities}
        return self.data

    def extract_data(self):

        self.paper_id = self.data['paper_id']
        self.title = self.data['title']
        self.abstract = self.data['abstract']
        self.text = self.data['text']
        self.abstract_tripples = self.data['abstract_tripples']
        self.text_tripples = self.data['text_tripples']
        self.key_phrases = self.data['key_phrases']
        self.entities = self.data['entities']

    def save(self, dir):
        self.combine_data()

        if not os.path.exists(os.path.dirname(dir)):
            try:
                os.makedirs(os.path.dirname(dir))
            except OSError as exc:  # Guard against race condition
                if exc.errno != errno.EEXIST:
                    raise

        with open(dir, 'w') as json_file:
            json_file.write(json.dumps(self.data))

    def load_saved_data(self, dir):
        with open(dir) as json_file:
            self.data = json.load(json_file)
        self.extract_data()


In [4]:
desired_dirs=['/kaggle/input/CORD-19-research-challenge/noncomm_use_subset/noncomm_use_subset',
             '/kaggle/input/CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv',
             '/kaggle/input/CORD-19-research-challenge/comm_use_subset/comm_use_subset',
             '/kaggle/input/CORD-19-research-challenge/custom_license/custom_license']


noncomm_use_subset=[]
biorxiv_medrxiv = []
comm_use_subset = []
custom_license = []

for individual_dirs in desired_dirs:
    files_list = []
    data = []
    print('############## Directory Working on #################')
    print(individual_dirs)
    print(individual_dirs.split('/')[-1])
    for dirname,_, filenames in os.walk(individual_dirs):
        #print(dirname)
        for filename in filenames:
            #print(os.path.join(dirname, filename))
            files_list.append(os.path.join(dirname, filename))
    print(len(files_list))
    #print(files_list)
    i=0
    for individual_file in files_list:
        try:
            pub = document(individual_file)
            data_dict = pub.clean_text()
            #print(data_dict)
            data.append(data_dict)
            i+=1
        except:
            pass
    print('Now writing back data')
    print('files processed===>'+str(i))
    with open(individual_dirs.split('/')[-1]+'.pickle', "wb") as f:
                pickle.dump(data,f)
    

############## Directory Working on #################
/kaggle/input/CORD-19-research-challenge/noncomm_use_subset/noncomm_use_subset
noncomm_use_subset
4678
Now writing back data
files processed===>4678
############## Directory Working on #################
/kaggle/input/CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv
biorxiv_medrxiv
1934
Now writing back data
files processed===>1934
############## Directory Working on #################
/kaggle/input/CORD-19-research-challenge/comm_use_subset/comm_use_subset
comm_use_subset
18746
Now writing back data
files processed===>18746
############## Directory Working on #################
/kaggle/input/CORD-19-research-challenge/custom_license/custom_license
custom_license
35238
Now writing back data
files processed===>35238


In [5]:
total_dataframe = pd.DataFrame()
models = glob.glob('/kaggle/working/' + "*.pickle")
#models = glob.glob('/kaggle/working/biorxiv_medrxiv.pickle')
# print('models via glob===>'+str(models))
for individual_model in models:
    print(individual_model)
    # open a file, where you stored the pickled data
    file = open(individual_model, 'rb')

    # dump information to that file
    data = pickle.load(file)

    # close the file
    file.close()

    print('Showing the pickled data:')
    my_df = pd.DataFrame(data)
    print(my_df.shape)
    my_df = my_df.replace(r'^\s*$', np.nan, regex=True)
    total_dataframe = total_dataframe.append(my_df, ignore_index=True)
    print('Null per column')
    print(my_df.isnull().sum())



/kaggle/working/noncomm_use_subset.pickle
Showing the pickled data:
(4678, 8)
Null per column
paper_id                0
title                 289
abstract             2901
text                  156
abstract_tripples       0
text_tripples           0
key_phrases          4678
entities                0
dtype: int64
/kaggle/working/custom_license.pickle
Showing the pickled data:
(35238, 8)
Null per column
paper_id                 0
title                 3712
abstract             17327
text                  1715
abstract_tripples        0
text_tripples            0
key_phrases          35238
entities                 0
dtype: int64
/kaggle/working/biorxiv_medrxiv.pickle
Showing the pickled data:
(1934, 8)
Null per column
paper_id                0
title                  78
abstract              251
text                    0
abstract_tripples       0
text_tripples           0
key_phrases          1934
entities                0
dtype: int64
/kaggle/working/comm_use_subset.pickle
Showing the pi

In [6]:
total_dataframe = total_dataframe.drop(['abstract_tripples', 'text_tripples','entities','key_phrases'], axis=1)

In [7]:
print('Total data in dataframe')
print(total_dataframe.shape)
total_dataframe.head()


Total data in dataframe
(60596, 4)


,paper_id,title,abstract,text
0,3cdc48bb9e40afd30a59463b7872761a726998c8,experimental evaluation of musca domestica as...,house ßies musca domestica l. were examined...,newcastle disease is an emerging disease affe...
1,d99acb4e99be7852aa61a688c9fbd38d44b5a252,evaluation of measles vaccine virus as a vecto...,live attenuated recombinant measles vaccine vi...,live attenuated viruses have been developed an...
2,748d4c57fe1acc8d9d97cf574f7dea5296f9386c,direct visualization of ebola virus fusion tri...,ebola virus makes extensive and intricate use...,occurs primarily through a macropinocytosis-li...
3,b891efc6e1419713b05ff7d89b26d260478c28df,NaN,china has the world's second largest tuberculo...,the goal of the present study was to investiga...
4,353852971069ad5794445e5c1ab6077ce23da75d,comment,NaN,coronavirus disease 2019 has spread with unpr...


In [8]:
total_dataframe.isnull().sum()

paper_id        0
title        4459
abstract    30684
text         1942
dtype: int64

In [9]:
tf_df = pd.DataFrame()
tf_df['merged_text'] = total_dataframe['title'].astype(str) +  total_dataframe['abstract'].astype(str) +  total_dataframe['text'].astype(str)
tf_df['paper_id'] = total_dataframe['paper_id']
tf_df.head()

,merged_text,paper_id
0,experimental evaluation of musca domestica as...,3cdc48bb9e40afd30a59463b7872761a726998c8
1,evaluation of measles vaccine virus as a vecto...,d99acb4e99be7852aa61a688c9fbd38d44b5a252
2,direct visualization of ebola virus fusion tri...,748d4c57fe1acc8d9d97cf574f7dea5296f9386c
3,nanchina has the world's second largest tuberc...,b891efc6e1419713b05ff7d89b26d260478c28df
4,commentnancoronavirus disease 2019 has spread...,353852971069ad5794445e5c1ab6077ce23da75d
